In [ ]:
import time

import numpy as np
from matplotlib import pyplot as plt
from scipy.io import loadmat
from skimage.io import imread

In [ ]:
rootfolder = ".."

## IRLS Algorithm

Define the problem parameters


In [ ]:
A = np.array([[1, 3], [3, 1]])  # low dimensions to plot it, you can test larger sizes
b = np.array([-1, 2])

lmbda = 0.5


The function to be minimized is $\frac{1}{2}\|Ax-b\|_2^2 + \lambda \|x\|_1$


In [ ]:
f = lambda x: 0.5 * np.sum((A @ x - b) ** 2) + lmbda * np.sum(np.abs(x))

# derivative of f from matrix calculus
df = lambda x: A.T @ (A @ x) - A.T @ b


Plot the function


In [ ]:
# this function has been prepared only for the visualization sake, no need to go through this but it renders some nice
# graphics :)
F = (
    lambda r1, r2: (r1 * A[0, 0] + r2 * A[0, 1] - b[0]) ** 2
    + (r1 * A[1, 0] + r2 * A[1, 1] - b[1]) ** 2
    + lmbda * (np.abs(r1) + np.abs(r2))
)
xx, yy = np.meshgrid(np.arange(-10, 10, 1), np.arange(-10, 10, 1))

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.plot_surface(xx, yy, F(xx, yy), edgecolor=[0, 0, 1], alpha=0.5, facecolor=[0, 1, 1])

Set the parameters


In [ ]:
MAX_ITER = 1e3
TOL_DIST_X = 1e-10

Initialization: test different inizializations, the function is convex, you always converge to the same solution


In [ ]:
x0 = np.array([5, -10])

# initialization
all_x = [x0]
distanceX = 1e10  # stopping criteria
cnt = 0
delta = 1e-6

Main loop


In [ ]:
while cnt < MAX_ITER and distanceX > TOL_DIST_X:
    x = all_x[-1]

    # compute the weight matrix
    W = np.diag(1 / (np.abs(x) + delta))

    # solve the weighted regularized LS system
    # (A^T A + lambda * W) x = A^T b
    x_current = np.linalg.solve(A.T @ A + lmbda * W, A.T @ b)

    # compute distance between consecutive iterates
    distanceX = np.linalg.norm(x_current - x)

    # store the estimate
    all_x.append(x_current.copy())

    cnt = cnt + 1

Plot all the estimates


In [ ]:
# plot the new estimate
xxplot = [x[0] for x in all_x]
yyplot = [x[1] for x in all_x]
zzplot = F(np.array(xxplot), np.array(yyplot))

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection="3d")
ax.plot_surface(xx, yy, F(xx, yy), edgecolor=[0, 0, 1], alpha=0.5, facecolor=[0, 1, 1])
ax.plot3D(xxplot, yyplot, zzplot, "r-o")

In [ ]:
print(f"nr of iteration of IRLS (before stopping criteria met): {cnt}\n")
print(f"Solution of IRLS: [{x_current[0]:.4f}, {x_current[1]:.4f}]\n")
print(f"Value of the functional: {f(x_current):.4f}\n")


## **THIS WAS NOT PRESENTED IN THE 2025 EDITION**: MOD dictionary learning

Useful function for plot the 2D DCT dictionary


In [ ]:
def get_dictionary_img(D):
    M, N = D.shape
    p = int(round(np.sqrt(M)))
    nnn = int(np.ceil(np.sqrt(N)))
    bound = 2
    img = np.ones((nnn * p + bound * (nnn - 1), nnn * p + bound * (nnn - 1)))
    for i in range(N):
        m = np.mod(i, nnn)
        n = int((i - m) / nnn)
        m = m * p + bound * m
        n = n * p + bound * n
        atom = D[:, i].reshape((p, p))
        if atom.min() < atom.max():
            atom = (atom - atom.min()) / (atom.max() - atom.min())
        img[m : m + p, n : n + p] = atom

    return img

Define a function to perform the sparse coding using your favorite algorithm (IRLS, FISTA or ISTA)


In [ ]:
def IRLS(s, D, lmbda, x0=None):
    # Parameters
    MAX_ITER = 100
    TOL_DIST_X = 1e-6
    delta = 1e-6

    # Initialize
    if x0 is None:
        x = np.zeros(D.shape[1])
    else:
        x = x0.copy()

    distanceX = 1e10
    cnt = 0

    while cnt < MAX_ITER and distanceX > TOL_DIST_X:
        # Store previous x for distance calculation
        x_prev = x.copy()

        # Compute the weight matrix
        W = np.diag(1 / (np.abs(x) + delta))

        # Solve the weighted regularized LS system
        # (D^T D + lambda * W) x = D^T s
        x = np.linalg.solve(D.T @ D + lmbda * W, D.T @ s)

        # Compute distance between consecutive iterates
        distanceX = np.linalg.norm(x - x_prev)

        cnt += 1

    return x

Load the image and rescale it in [0,1]


In [ ]:
img = imread(f"{rootfolder}/data/barbara.png") / 255
imsz = img.shape


Set the parameters


In [ ]:
# patch size
p = 8

# number of elements in the patchfrom skimage.io import imread
M = p**2

# number of columns in the dictionary
N = 96

# extract the random patches from the noisy image
npatch = 1000

# only few MOD iterations are needed for a good dictionary
max_iter = 10

lmbda = 0.1

Extract $npatch$ random patches from the image


In [ ]:
S = np.zeros((M, npatch))
for n in range(npatch):
    # Generate random coordinates
    i = np.random.randint(0, imsz[0] - p + 1)
    j = np.random.randint(0, imsz[1] - p + 1)
    # Extract patch and vectorize it
    patch = img[i : i + p, j : j + p]
    S[:, n] = patch.flatten()

Initialize the dictionary randomly and the normalize the columns


In [ ]:
D = np.random.randn(M, N)
# Normalize columns to unit norm
D = D / np.linalg.norm(D, axis=0)

Initialize a matrix for the coefficients of all the patches


In [ ]:
X = np.zeros((N, npatch))

Main loop


In [ ]:
for iter in range(max_iter):
    # perform the sparse coding for all the patches in S
    for n in range(npatch):
        s = S[:, n]
        x = IRLS(s, D, lmbda)
        X[:, n] = x

    # MOD update: solve D = S * X^T * (X * X^T)^(-1)
    D = S @ X.T @ np.linalg.pinv(X @ X.T)

    # normalize the columns
    D = D / np.linalg.norm(D, axis=0)

Show the dictionary


In [ ]:
img_dict = get_dictionary_img(D)
plt.figure()
plt.imshow(img_dict, cmap="gray")


## Denoising via $\ell^1$ sparse coding (use a dictionary learned by KSVD)

Set the noise level and add the noise to the original image


In [ ]:
sigma_noise = 20 / 255
noisy_img = img + np.random.normal(size=imsz) * sigma_noise


Compue the psnr of the noisy input


In [ ]:
# Compute PSNR of noisy input
psnr_noisy = 10 * np.log10(1 / np.mean((noisy_img - img) ** 2))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img, cmap="gray")
ax[0].set_title("Original image")

ax[1].imshow(noisy_img, cmap="gray")
ax[1].set_title(f"Noisy image, PSNR = {psnr_noisy:.2f}")

Use the dictionary computed with the MOD or load a pretrained dictionary $D$


In [ ]:
def OMP(s, D, L, tau):
    _, N = D.shape
    r = s.copy()  # initial residual
    omega = []  # support set
    x_OMP = np.zeros(N)  # final sparse code

    while len(omega) < L and np.linalg.norm(r) > tau:
        # SWEEP STEP: compute correlations between residual and dictionary atoms
        e = np.zeros(N)
        for j in range(N):
            e[j] = D[:, j].T @ r

        # find the column index with maximum correlation
        jStar = np.argmax(np.abs(e))

        # UPDATE support set
        if jStar not in omega:
            omega.append(jStar)

        # update coefficients using least squares
        D_omega = D[:, omega]
        x_omega, _, _, _ = np.linalg.lstsq(D_omega, s, rcond=None)

        # update residual
        r = s - D_omega @ x_omega

    # construct full sparse vector
    for i, idx in enumerate(omega):
        x_OMP[idx] = x_omega[i]

    return x_OMP

In [ ]:
def ksvd(S, M, N, max_iter, npatch, L, print_time=False):
    # initialize the dictionary
    D = np.random.randn(M, N)

    # normalize each column of D (zero mean and unit norm)
    # UPDATE D
    D = D - np.mean(D, axis=0, keepdims=True)
    D = D / np.linalg.norm(D, axis=0, keepdims=True)

    # initialize the coefficient matrix
    X = np.zeros((N, npatch))

    # Main KSVD loop
    for iter in range(max_iter):
        time_start = time.time()

        # Sparse coding step
        # perform the sparse coding via OMP of all the columns of S
        for n in range(npatch):
            X[:, n] = OMP(S[:, n], D, L, 1e-6)

        # Dictionary update step
        # iterate over the columns of D
        for j in range(N):
            # find which signals uses the j-th atom in the sparse coding
            omega = np.where(X[j, :] != 0)[0]

            if len(omega) == 0:
                # if the atom is never used then ignore or substitute it with a random vector
                D[:, j] = np.random.randn(M)
                D[:, j] = D[:, j] / np.linalg.norm(D[:, j])
            else:
                # compute the residual matrix E, ignoring the j-th atom
                E = S - D @ X + np.outer(D[:, j], X[j, :])

                # restrict E to the columns indicated by omega
                Eomega = E[:, omega]

                # Compute the best rank-1 approximation
                U, Sigma, Vt = np.linalg.svd(Eomega, full_matrices=False)

                # update the dictionary
                D[:, j] = U[:, 0]

                # update the coefficient matrix
                X[j, omega] = Sigma[0] * Vt[0, :]

        time_end = time.time()
        if print_time:
            print(f"Iteration {iter} runtime: {time_end - time_start}")

    return D

In [ ]:
D = loadmat(f"{rootfolder}/data/dict_nat_img.mat")["D"]

# show the dictionary
D_img = get_dictionary_img(D)
plt.figure(figsize=(10, 10))
plt.imshow(D_img, cmap="gray")


In [ ]:
# initialize the estimated image
img_hat = np.zeros_like(img)

# initialize the weight matrix
weights = np.zeros_like(img)

# set the threshold
tau = 2.2
lmbda = tau * sigma_noise

# define the step (=p for non overlapping paches)
STEP = 4  # STEP = 1 might be very time consuming, start with larger STEP

Operate patchwise


In [ ]:
for i in range(0, imsz[0] - p + 1, STEP):
    for j in range(0, imsz[1] - p + 1, STEP):
        # extract the patch with the top left corner at pixel (i, j)
        s = noisy_img[i : i + p, j : j + p].flatten()

        # store and subtract the mean
        s_mean = np.mean(s)
        s = s - s_mean

        # perform the sparse coding of the patch s to compute the coefficients vector x
        x = IRLS(s, D, lmbda)

        # perform the reconstruction
        s_hat = D @ x

        w = 1

        # add back the mean
        s_hat = s_hat + s_mean

        # put the denoised patch into the estimated image using uniform weights
        img_hat[i : i + p, j : j + p] += w * s_hat.reshape((p, p))

        # store the weight of the current patch in the weight matrix
        weights[i : i + p, j : j + p] += w

Normalize the estimated image with the computed weights


In [ ]:
img_hat = img_hat / (weights + 1e-10)  # Add small epsilon to avoid division by zero

Compute the psnr of the estimated image


In [ ]:
psnr_hat = 10 * np.log10(1 / np.mean((img - img_hat) ** 2))
plt.figure(figsize=(10, 10))
plt.imshow(img_hat, cmap="gray")
plt.title(f"Estimated Image,\nPSNR = {psnr_hat:.2f}")